In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import time
import threading
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import HBox, VBox, FloatSlider, ToggleButton


# max values for NX and NY
MAXX = 128
MAXY = 128

# Horiz and vert number of lattice points, and total number of spins
NX = 128
NY = 128
NSPIN = NX*NY

# Number of thermalization sweeps
#NTHERM = 3
NTHERM = 400

# Freq of sweeps to avoid correlations
NFREQ = 100
#NFREQ = 1

# Number of entries in averages (each entry separated by NFREQ sweeps)
#NSIZE = 1000
NSIZE = 100

In [ ]:
def checkerboard(shape):
    """
    Thank you stackoverflow!
    """
    return np.indices(shape).sum(axis=0) % 2

In [ ]:
def make_random_spin_array(nx, ny):
    rand_vals = np.random.rand(nx, ny)
    ones = np.ones((nx, ny), dtype=int)
    rslt = np.choose(rand_vals >= 0.5,
                     [-ones, ones])
    return rslt

In [ ]:
class MetropolisRatio:
    def __init__(self):
        self.array = np.zeros((3,9))
    def set(self, nbr_sum, this_spin, value):
        self.array[this_spin+1, nbr_sum+4] = value
    def get(self, nbr_sum, this_spin):
        return self.array[this_spin+1, nbr_sum+4]
    def dump(self):
        print(self.array)

In [ ]:
def count_neighbors_and_pick_flips(spins, ratios):
    """
    Returns a boolean array which is true for spins that would flip,
    over the entire array (not just the red or black squares)
    """
    nbr_counts = (np.roll(spins, 1, 0)
                  + np.roll(spins, -1, 0)
                  + np.roll(spins, 1, 1)
                  + np.roll(spins, -1, 1)
    )
    flip_probs = ratios.get(nbr_counts, spins)
    does_this_spin_flip = (np.random.rand(*spins.shape) <= flip_probs)
    return does_this_spin_flip

In [ ]:
class IsingSystem:
    def __init__(self, nx, ny, b=0.0, j=0.4):
        self.nx = nx
        self.ny = ny
        self.nspins = nx * ny
        self.b = b
        self.j = j
        self.ratios = MetropolisRatio()
        self.update_ratios()
        self.spins = make_random_spin_array(self.nx, self.ny)
        self.checkerboard_red = checkerboard(self.spins.shape)
        self.checkerboard_black = 1 - self.checkerboard_red
    def update_ratios(self):
        """Recalculate Metropolis ratios when b or j changes"""
        j = self.j
        b = self.b
        for nnbrloop in range(-4, 5, 2):   # -4, -2, 0, 2, 4
            self.ratios.set(nnbrloop, -1, np.exp(2.0*(j*nnbrloop + b)))  # SOME FORMULA GOES HERE
            self.ratios.set(nnbrloop, 1, np.exp(-2.0*(j*nnbrloop + b)))  # SOME FORMULA GOES HERE
    def dump(self):
        print('Describing IsingSystem:')
        print(f'  nx = {self.nx} ny = {self.ny} b = {self.b} j = {self.j}')
        print(' ratios:')
        self.ratios.dump()
        print(' spins:')
        print(self.spins)
        print(' checkerboard_red:')
        print(self.checkerboard_red)
    def metrop(self):
        """ 
        Updates the spins based on the given ratios.  Returns
        the fraction of spins that were updated.
        """
        # For the 'red' squares first, flip appropriate spins
        does_this_spin_flip = count_neighbors_and_pick_flips(self.spins, self.ratios)
        does_this_spin_flip = np.logical_and(does_this_spin_flip,
                                             self.checkerboard_red)
        num_flips = np.count_nonzero(does_this_spin_flip)
        self.spins = np.choose(does_this_spin_flip, [self.spins, -self.spins])

        # Now do the 'black' squares
        does_this_spin_flip = count_neighbors_and_pick_flips(self.spins, self.ratios)
        does_this_spin_flip = np.logical_and(does_this_spin_flip, self.checkerboard_black)
        num_flips += np.count_nonzero(does_this_spin_flip)
        self.spins = np.choose(does_this_spin_flip, [self.spins, -self.spins])

        return num_flips/self.nspins
    def measure_physical_quantities(self):
        """
        Given the spin matrix and the current values of b and j,
        returns magnetization and total system energy, in that order
        """
        # Take care to count each pairing only once
        pairs = np.sum(self.spins * (np.roll(self.spins, -1, 0) + np.roll(self.spins, -1, 1)))
        magsweep = np.sum(self.spins) / self.nspins
        esweep = (-self.j * pairs - self.b * np.sum(self.spins)) / self.nspins
        return magsweep, esweep        
    def measure(self, nfreq, nsamples):
        """
        nfreq is the number of updates between samples (to avoid correlation)
        nsamples is the number of samples to include in the measurement.
        """
        energy = 0.0  # running total of energy
        mag = 0.0  # running total of magnetization
        nmeasure = 0  # how many measurements have been made
        for iter in range(nfreq * nsamples):
            accept = self.metrop()
            if (iter % nfreq == 0):
                magsweep, esweep = self.measure_physical_quantities()
                mag += np.abs(magsweep)
                energy += esweep
                nmeasure += 1
                print(f'     Measurement number={nmeasure}, energy={esweep}, '
                      f'magnetization={magsweep}')


In [ ]:
ising_system = IsingSystem(128,128)
ising_system.dump()
ising_system.measure(100, 10)

In [ ]:
class IsingWidget:
    def __init__(self, ising_system, cmap_name='viridis'):
        self.ising_system = ising_system
        self.cmap_name = cmap_name
        self.fig, self.ax = plt.subplots()
        self.fig.set_size_inches(3.0, 3.0)
        self.fig.canvas.toolbar_position = "bottom"
        self.im = None
        self.run = False
        self.bg_thread = None
        self.redraw()
    def redraw(self, cmap_name=None):
        if cmap_name is None:
            cmap_name = self.cmap_name
        if self.im is None:
            self.im = self.ax.imshow(self.ising_system.spins, cmap=cmap_name,
                                     origin="upper", aspect="equal")
        else:
            self.im.set_cmap(self.cmap_name)
            self.im.set_data(self.ising_system.spins)
    def work_loop(self):
        while self.run:
            self.ising_system.metrop()
            self.redraw()
            time.sleep(0.000001) # yield the thread
    def j_observer(self, j_bunch):
        if j_bunch['name'] == 'value':
            self.ising_system.j = j_bunch.new
            self.ising_system.update_ratios()
            self.redraw()
    def run_observer(self, bool_bunch):
        if bool_bunch['name'] == 'value':
            if bool_bunch.new:
                self.run = True
                self.bg_thread = threading.Thread(target=self.work_loop).start()
            else:
                self.run = False
                self.bg_thread = None


In [ ]:
output = widgets.Output()
with output:
    ising_widget = IsingWidget(ising_system)
j_slider = FloatSlider(0.4, min=0.0, max=1.0)
j_slider.observe(ising_widget.j_observer)
run_button = ToggleButton(value=False, description="Run", icon="check")
run_button.observe(ising_widget.run_observer)

In [ ]:
HBox([VBox([j_slider, run_button]), output])